In [21]:
import networkx as nx
from networkx.generators import community as comm
import igraph as ig
import powerlaw
import numpy as np
import matplotlib.pyplot as plt

In [22]:
def import_nx_network(net):
    graph = ig.Graph(n=net.number_of_nodes(), directed=False)
    graph.add_edges(net.edges())

    return graph

def LFR_graph(N, τ1, τ2, μ, **kwargs):
     net = comm.LFR_benchmark_graph(N, τ1, τ2, μ, **kwargs)
     graph = import_nx_network(net)

     return graph

In [23]:
G = nx.DiGraph()

In [24]:
# get edgelist from this file as string '../data/hashtags/edges/lgbt_edges.txt'
edgelist = open('../data/hashtags/edges/lgbt_edges.txt').read()

In [25]:
for line in edgelist.strip().split('\n'):
    node1, node2 = line.split(',')
    if node2 != 'None':  # Ignore edges where the second node is 'None'
        G.add_edge(node1, node2)

In [26]:
degree_sequence = sorted((d for n, d in G.in_degree()), reverse=True)

In [27]:
fit = powerlaw.Fit(degree_sequence)

Calculating best minimal value for power law fit


Values less than or equal to 0 in data. Throwing out 0 or negative values


In [28]:
tau1 = fit.power_law.alpha

In [29]:
tau1

11.03843762418175

In [30]:
from networkx.algorithms import community as community
communities = community.greedy_modularity_communities(G)
partition = {}
for i, comm in enumerate(communities):
    for node in comm:
        partition[node] = i
total_mu = 0
for node in G.nodes():
    # Get the community of the current node
    community_of_node = partition[node]
    
    # Count intra-community edges (within the same community)
    intra_community_edges = sum(1 for neighbor in G.neighbors(node) if partition[neighbor] == community_of_node)
    
    # Total number of edges for this node (degree)
    total_edges = G.degree[node]
    
    # Compute inter-community edges
    inter_community_edges = total_edges - intra_community_edges
    
    # Compute mu for this node
    if total_edges > 0:
        mu_node = inter_community_edges / total_edges
    else:
        mu_node = 0  # Handle isolated nodes
    
    # Accumulate mu
    total_mu += mu_node

# Compute the average mu for the graph
mu = total_mu / G.number_of_nodes()
mu

0.4396020843202274

In [31]:
community_sizes = [list(partition.values()).count(c) for c in set(partition.values())]

In [32]:
fit_community = powerlaw.Fit(community_sizes)
tau2 = fit_community.power_law.alpha
tau2

Calculating best minimal value for power law fit


16.199971734892827

In [35]:
from networkx.generators import community as comm
n = 1685
average_degree = 1170/1685
tau1 = 11.03843762418175
tau2 = 16.199971734892827
mu = 0.4396020843202274
G = comm.LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=average_degree, seed=10, max_iters=1000
)

ExceededMaxIterations: Could not match average_degree